# BINF TP3 - Algorithmes d'alignement par paire

Dans ce TP nous allons manipuler les algorithmes d'alignement par paire.

# Exercice 0 - Echauffement

Q1. Donnez le score de la superposition :

|       |       |
| :---: | :---: |
x       | ATGTCATGA---TAC |
y       | AT--CTAAATGTTAC |


étant donne le schéma d'évaluation :

|       | A     | T     | G     | C     |
| :---: | :---: | :---: | :---: | :---: |
| **A** | 1     | -1    | -1    | -1    |
| **T** | -1    | 1     | -1    | -1    |
| **G** | -1    | -1    | 1     | -1    |
| **C** | -1    | -1    | -1    | 1     |

et

$\gamma(g) = 0.5 |g| + 0.5$

```markdown
On compare caractère par caractère les 2 séquences et à partir de cela on applique la matrice de substitution. Ainsi lorsqu'il y a un match identique on va ajouter 1 et pour un mismatch on va retirer un et pour les gaps on va calculer la longueur du gap et appliquer la fonction des trous. Un gap de longueur 2 va donner gamma(2) = 0.5 * 2 + 0.5 = 1.5 et un gap de longueur 3 va donner gamma(3) = 0.5 * 3 + 0.5 = 2.
Ce qui va nous donner : 1 + 1 - 1.5 + 1 - 1 - 1 - 1 + 1 - 2 + 1 + 1 + 1 = 0.5
```

Q2. Alignez les séquences suivantes avec l'algorithme de Levenshtein :  x = ATG et y = ACTG.

```markdown
Si d'abord on calcule la distance de Levenshtein sur les 2 séquences suivantes on aura 1 car on a juste à ajouter un C au premier caractère du premier mot.
```

Q3.	Alignez les séquences suivantes avec l'algorithme de Needleman-Wunsch global x = TAT et y = ATGAC en considérant le schéma d'évaluation suivant

|       | A       | T       | G       | C     |
| :---: | :---:   | :---:   | :---:   | :---: |
| **A** | 1       | -0.5    | -0.5    | -0.5  |
| **T** | -0.5    | 1       | -0.5    | -0.5  |
| **G** | -0.5    | -0.5    | 1       | -0.5  |
| **C** | -0.5    | -0.5    | -0.5    | 1     |

et

$\gamma(g) = 0.5 |g|$


```markdown
On rempli tout d'abord la matrice en utilisant le critère de remplissage puis lorsque la matrice est totalement remplie on remonte la matrice de backtrack. Ainsi on trouve x = T-AT et y = TGAC.
```

Q4. Alignez les séquences suivantes avec l'algorithme de Smith-Waterman x = TTGG y = ATGAC en utilisant le schéma d'évaluation de la question précédente.


```markdown
On rempli la matrice de en utilisant le critère de remplissage avec en parallèle la matrice de backtrack et ensuite on remonte la matrice de backtrack. Ainsi on trouve un score optimal de 2.5.
```

# Exercice 1 : Algorithme de Levenshtein - version récursive

Q1. Ecrivez une fonction

levenshtein(x: str, y: str) -> int

qui retourne la distance de Levenshtein entre les séquences x et y en utilisant la  version récursive de l'algorithme.

In [1]:
def levenshtein_rec(x, y, m, n):
    if (m == 0):
        return n
    if n == 0:
        return m
    
    if (x[m-1] == y[n-1]):
        return levenshtein_rec(x, y, m-1,n-1)
    return 1 + min(
        levenshtein_rec(x, y, m, n-1),
        min(
            levenshtein_rec(x, y, m-1, n),
            levenshtein_rec(x, y, m-1,n-1)
        )
    )

In [2]:
import numpy as np

UP = 1
LEFT = 2
DIAG = 3

def levenshtein_iter(x, y):
    m, n = len(x), len(y)
 
    # Matrices
    S = np.zeros((m+1, n+1))  # Matrice des scores
    B = np.zeros((m+1, n+1), dtype=int)  # Matrice des directions
    
    # Initialisation des bords
    for i in range(1, m+1):
        S[i, 0] = i
        B[i, 0] = UP
    for j in range(1, n+1):
        S[0, j] = j
        B[0, j] = LEFT
    
    # Remplissage des matrices
    for i in range(1, m+1):
        for j in range(1, n+1):
            cost = 0 if x[i-1] == y[j-1] else 1

            diag = S[i-1, j-1] + cost
            up = S[i-1, j] + 1
            left = S[i, j-1] + 1

            S[i, j] = min(diag, up, left)

            # Stockage de la direction optimale
            if S[i, j] == diag:
                B[i, j] = DIAG
            elif S[i, j] == up:
                B[i, j] = UP
            else:
                B[i, j] = LEFT
    
    # Backtrack
    i, j = m, n
    sx, sy = "", ""

    while (i, j) != (0, 0):
        if B[i, j] == DIAG:
            sx = x[i-1] + sx
            sy = y[j-1] + sy
            i, j = i-1, j-1
        elif B[i, j] == UP:
            sx = x[i-1] + sx
            sy = "-" + sy
            i -= 1
        else:  # LEFT
            sx = "-" + sx
            sy = y[j-1] + sy
            j -= 1

    return S[m, n], sx, sy

In [6]:
val = levenshtein_rec('CCAG', 'CA', len('CCAG'), len('CA'))
print(val)

val = levenshtein_rec('CCGT', 'CGTCA', len('CCGT'), len('CGTCA'))
print(val)

val, sx, sy = levenshtein_iter("ATGGTGAGCAAGGGCGAGGAGGATAACATGGCCATCATCAAGGAGTTCATGCGCTTCAAGGTGCACATGGAGGGCTCCGTGAACGGCCACGAGTTCGAGATCGAGGGCGAGGGCGAGGGCCGCCCCTACGAGGGCACCCAGACCGCCAAGCTGAAGGTGACCAAGGGTGGCCCCCTGCCCTTCGCCTGGGACATCCTGTCCCCTCAGTTCATGTACGGCTCCAAGGCCTACGTGAAGCACCCCGCCGACATCCCCGACTACTTGAAGCTGTCCTTCCCCGAGGGCTTCAAGTGGGAGCGCGTGATGAACTTCGAGGACGGCGGCGTGGTGACCGTGACCCAGGACTCCTCCCTGCAGGACGGCGAGTTCATCTACAAGGTGAAGCTGCGCGGCACCAACTTCCCCTCCGACGGCCCCGTAATGCAGAAGAAGACCATGGGCTGGGAGGCCTCCTCCGAGCGGATGTACCCCGAGGACGGCGCCCTGAAGGGCGAGATCAAGCAGAGGCTGAAGCTGAAGGACGGCGGCCACTACGACGCTGAGGTCAAGACCACCTACAAGGCCAAGAAGCCCGTGCAGCTGCCCGGCGCCTACAACGTCAACATCAAGTTGGACATCACCTCCCACAACGAGGACTACACCATCGTGGAACAGTACGAACGCGCCGAGGGCCGCCACTCCACCGGCGGCATGGACGAGCTGTACAAGTAA", "ATGGTCTCCTTCAAATCTCTCCTAGTTCTCTGTTGCGCTGCCCTTGGGGCATTCGCTACGAAGAGAATGAGTAAAGGAGAAGAACTTTTCACTGGAGTTGTCCCAATTCTTGTTGAATTAGATGGTGATGTTAATGGGCACAAATTTTCTGTCAGTGGAGAGGGTGAAGGTGATGCAACATACGGAAAACTTACCCTTAAATTTATTTGCACTACTGGAAAACTACCTGTTCCATGGCCAACACTTGTCACTACTTTCACCTATGGTGTTCAATGCTTTTCAAGATACCCAGATCATATGAAGCGGCACGACTTCTTCAAGAGCGCCATGCCTGAGGGATACGTGCAGGAGAGGACCATCTTCTTCAAAGACGACGGGAACTACAAGACACGTGCTGAAGTCAAGTTTGAGGGAGACACCCTCGTCAACAGGATCGAGCTTAAGGGAATCGATTTCAAGGAGGACGGAAACATCCTCGGCCACAAGTTGGAATACAACTACAACTCCCACAACGTATACATCATGGCCGACAAGCAAAAGAACGGCATCAAAGCCAACTTCAAGACCCGCCACAACATCGAAGACGGCGGCGTGCAACTCGCTGATCATTATCAACAAAATACTCCAATTGGCGATGGCCCTGTCCTTTTACCAGACAACCATTACCTGTCCACACAATCTGCCCTTTCGAAAGATCCCAACGAAAAGAGAGACCACATGGTCCTTCTTGAGTTTGTAACAGCTGCTGGGATTACACATGGCATGGATGAACTATACAAATAA")
print(val, " ", sx, " ", sy)

2
3
391.0   ATGG--T-GAGC-AA---GGGCGAG------G-AG-GAT-AACAT-GGCCA-TC-AT-C--A-AG---GAGTTCA--TG-CG--C--TTCA----AG--GT-GC-A--CATG--G----AG--GGCTCCGTGAACGGCCACGAGTTCGAGATCGAG-GGCGAGGGCG-AGG-GCCGCCCCTACGAGGGCACCCAGACCGCCAAGCTGAAGGTG-ACCA-AGGGTGGCCCCCTG-CCCTTCGCCTGGGACATCCTGTCCCCTCAGTTCATGTACGG-CTCCAAGGCCTACGTGAAG-CACCCCG--CCGA--CATC--CCCGACTACTTGAAGCTGTCCTTCCCCGAGGGCTTCAAGTGGGAGCGCGTGATGAACTTCGAGGACGGCGGCGTGGTGAC-CGTGAC--CCAGGACTCCTCCCTGCAGGACGGCGAGTTCATCTACAAGGTGAAGCTGCGCGGCACCAACTTCCCCTCCGACGGCCCCGTAAT--G-CAGAAG-AAGACCATGGGCTGGGAGGCCTCCTCCG-A-GC-GGATGTACCCCGAG--GACG-GCGCCCTGAAGGGCGAGATCAAGCAGAGGCTGAAGCT-GAAGGACGGCGGC-CACTACGACGCTGAGGTCAAGACCACCTACA-AGGCCAA-----GA-AGCCC-GTGCAGCTGCCCGGC-GCC--TACAACGTCAACATCAAGTTGGAC-ATC---ACCTCCC-AC--AACGAG-GACTACACCATCGTGGAACAGTACG-AACGCGCCGAGGGCCGCCACTCCACCGGCGGCATGGACGAGCTGTACAAGTAA   ATGGTCTCCTTCAAATCTCTCCTAGTTCTCTGTTGCGCTGCCCTTGGGGCATTCGCTACGAAGAGAATGAGTAAAGGAGAAGAACTTTTCACTGGAGTTGTCCCAATTCTTGTTGAATTAGATGGTGATGTTAATGGGCACAAATTTTCTGTC-AGTGGAGAGGGTGAAGGTGA

Q2. Vous pouvez tester votre code sur les exemples suivants:


*   $L('CCAG', 'CA') = 2$
*   $L('CCGT', 'CGTCA') = 3$
*   $L(AY678264^*, OQ870305^*) = 310$

$^*$ ids genbank de deux sequences.

# Exercice 2 : Algorithme de Smith-Waterman - version itérative

Q1. Ecrivez la fonction

sw_fwd(x: str, y: str, cmap: dict, sigma: array, (go, ge): list) -> (array, array)

qui construit les matrices $S$ et $B$ en utilisant l'algorithme de Smith-Waterman pour aligner les séquences x et y suivant le schéma d'évaluation donné par la matrice de substitution $\Sigma$ et la fonction d'évaluation des trous $\gamma(n)= g_o + g_e \times n$. Le dictionnaire cmap donne la position des différents nucléotides dans la matrice $\Sigma$. La fonction retourne la paire de matrices de score $S$ et de retour $B$.

In [39]:
UP = 1
LEFT = 2
DIAG = 3

def sw_fwd(x, y, cmap, sigma, gap_params):
    m, n = len(x), len(y)
    
    # Matrices
    S = np.zeros((m+1, n+1))
    B = np.zeros((m+1, n+1))

    go, ge = gap_params
    
    # Initialiser la première ligne et la première colonne
    for i in range(1, m + 1):
        S[i, 0] = -go - ge * (i)
        B[i, 0] = UP

    for j in range(1, n + 1):
        S[0, j] = -go - ge * (j)
        B[0, j] = LEFT
    
    # Remplir les matrices S et B
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            match = S[i - 1, j - 1] + sigma[cmap[x[i - 1]], cmap[y[j - 1]]]
            delete = max([S[i - k, j] - (go + ge * (k - 1)) for k in range(1, i + 1)])
            insert = max([S[i, j - k] - (go + ge * (k - 1)) for k in range(1, j + 1)])
            
            S[i, j] = max(0, match, delete, insert)
            
            if S[i, j] == 0:
                B[i, j] = 0
            elif S[i, j] == match:
                B[i, j] = DIAG  # Diagonale (match ou mismatch)
            elif S[i, j] == delete:
                B[i, j] = UP  # Suppression (gap dans y)
            else:
                B[i, j] = LEFT  # Insertion (gap dans x)

    return S, B

Q2. Ecrivez la fonction

sw_bwd(x: str, y: str, S: array, B: array) -> (str, str, float)

qui effectue l'etape de retour de l'algorithme de Smith-Waterman etant donné les séquences $x$ et $y$ et les matrices de score $S$ et de retour $B$. La fonction retourne un tuple contenant les alignements des séquences x et y et le score de l'alignement.

In [40]:
def sw_bwd(x, y, S, B):
    # Trouver la position du score max
    i, j = np.unravel_index(np.argmax(S), S.shape)
    sx, sy = "", ""

    while S[i, j] > 0:
        if B[i, j] == DIAG:
            sx = x[i-1] + sx
            sy = y[j-1] + sy
            i, j = i-1, j-1
        elif B[i, j] == UP:
            sx = x[i-1] + sx
            sy = "-" + sy
            i -= 1
        elif B[i, j] == LEFT:
            sx = "-" + sx
            sy = y[j-1] + sy
            j -= 1
    
    # Score de l'alignement
    val = S[np.unravel_index(np.argmax(S), S.shape)]
    return sx, sy, val

Q3. Vous pouvez tester votre code en utilisant le schéma d'évaluation suivant :

In [51]:
cmap = {"A": 0, "T": 1, "G": 2, "C": 3}
m = np.array([[1, -0.5, -0.5, -0.5],
              [-0.5, 1, -0.5, -0.5],
              [-0.5, -0.5, 1, -0.5],
              [-0.5, -0.5, -0.5, 1]])
go = 0
ge = 0.5

# Exemple de séquences
x = "TTGG"
y = "ATGAC"

#x = "TCGC"
#y = "CTTAG"

# Calcul des matrices S et B
S, B = sw_fwd(x, y, cmap, m, [go, ge])

# Traçage arrière
sx, sy, val = sw_bwd(x, y, S, B)

# Affichage des résultats
print("Score Matrix (S):\n", S)
print("Backtrack Matrix (B):\n", B)
print("Aligned Sequences:\n", sx, "\n", sy)
print("Alignment Score:\n", val)


x = "ATGGTGAGCAAGGGCGAGGAGGATAACATGGCCATCATCAAGGAGTTCATGCGCTTCAAGGTGCACATGGAGGGCTCCGTGAACGGCCACGAGTTCGAGATCGAGGGCGAGGGCGAGGGCCGCCCCTACGAGGGCACCCAGACCGCCAAGCTGAAGGTGACCAAGGGTGGCCCCCTGCCCTTCGCCTGGGACATCCTGTCCCCTCAGTTCATGTACGGCTCCAAGGCCTACGTGAAGCACCCCGCCGACATCCCCGACTACTTGAAGCTGTCCTTCCCCGAGGGCTTCAAGTGGGAGCGCGTGATGAACTTCGAGGACGGCGGCGTGGTGACCGTGACCCAGGACTCCTCCCTGCAGGACGGCGAGTTCATCTACAAGGTGAAGCTGCGCGGCACCAACTTCCCCTCCGACGGCCCCGTAATGCAGAAGAAGACCATGGGCTGGGAGGCCTCCTCCGAGCGGATGTACCCCGAGGACGGCGCCCTGAAGGGCGAGATCAAGCAGAGGCTGAAGCTGAAGGACGGCGGCCACTACGACGCTGAGGTCAAGACCACCTACAAGGCCAAGAAGCCCGTGCAGCTGCCCGGCGCCTACAACGTCAACATCAAGTTGGACATCACCTCCCACAACGAGGACTACACCATCGTGGAACAGTACGAACGCGCCGAGGGCCGCCACTCCACCGGCGGCATGGACGAGCTGTACAAGTAA"
y = "ATGGTCTCCTTCAAATCTCTCCTAGTTCTCTGTTGCGCTGCCCTTGGGGCATTCGCTACGAAGAGAATGAGTAAAGGAGAAGAACTTTTCACTGGAGTTGTCCCAATTCTTGTTGAATTAGATGGTGATGTTAATGGGCACAAATTTTCTGTCAGTGGAGAGGGTGAAGGTGATGCAACATACGGAAAACTTACCCTTAAATTTATTTGCACTACTGGAAAACTACCTGTTCCATGGCCAACACTTGTCACTACTTTCACCTATGGTGTTCAATGCTTTTCAAGATACCCAGATCATATGAAGCGGCACGACTTCTTCAAGAGCGCCATGCCTGAGGGATACGTGCAGGAGAGGACCATCTTCTTCAAAGACGACGGGAACTACAAGACACGTGCTGAAGTCAAGTTTGAGGGAGACACCCTCGTCAACAGGATCGAGCTTAAGGGAATCGATTTCAAGGAGGACGGAAACATCCTCGGCCACAAGTTGGAATACAACTACAACTCCCACAACGTATACATCATGGCCGACAAGCAAAAGAACGGCATCAAAGCCAACTTCAAGACCCGCCACAACATCGAAGACGGCGGCGTGCAACTCGCTGATCATTATCAACAAAATACTCCAATTGGCGATGGCCCTGTCCTTTTACCAGACAACCATTACCTGTCCACACAATCTGCCCTTTCGAAAGATCCCAACGAAAAGAGAGACCACATGGTCCTTCTTGAGTTTGTAACAGCTGCTGGGATTACACATGGCATGGATGAACTATACAAATAA"

# Calcul des matrices S et B pour les longues séquences
S, B = sw_fwd(x, y, cmap, m, [go, ge])

# Traçage arrière pour les longues séquences
sx, sy, val = sw_bwd(x, y, S, B)

# Affichage des résultats pour les longues séquences
print("Aligned Sequences (Long):\n", sx, "\n", sy)
print("Alignment Score (Long):\n", val)

Score Matrix (S):
 [[ 0.  -0.5 -1.  -1.5 -2.  -2.5]
 [-0.5  0.   0.5  0.5  0.5  0.5]
 [-1.   0.   1.   1.   1.   1. ]
 [-1.5  0.   1.   2.   2.   2. ]
 [-2.   0.   1.   2.   2.   2. ]]
Backtrack Matrix (B):
 [[0. 2. 2. 2. 2. 2.]
 [1. 0. 3. 2. 2. 2.]
 [1. 0. 3. 2. 2. 2.]
 [1. 0. 1. 3. 2. 2.]
 [1. 0. 1. 3. 1. 1.]]
Aligned Sequences:
 TG 
 TG
Alignment Score:
 2.0
Aligned Sequences (Long):
 ATGG------------------TGAG----C-AAG--G-GC-G-----AGGAGG-A-----TA--ACATG-G-CCAT-CA-T-CAA-G-G-AG-----TTCA-T-G-CG-CT-T--CAA-----G-GTGCACA-T-GGA-GG-G-CTCCG-TGAA--CGGC-C-ACGA--GTTC-GAGATCGAG-GG-CGAGGG-CG-AGG-G--CCGC--C-CCTAC-G---AGGGC--ACCC--------A---G-AC--C--G--CCAAGCTGA---AG-GTGACCAAGGGTGGCC--C-C-CTG-C-C--C-TTC-GCC--TGG-G---ACAT-C----C--TG-T-CCC---CTCAGTTCATG-TA-CGGC-TC-CAAGGC--C-TACGTGAAGCA-C-CC--CGCC-GA---CAT-C---C--CCGA-CTA-CTTGA--A-GCTGTCCTTC-----CCCGA-GGG--CT-TCAAGTGGGAGC-GCGTG-ATGAA-CTTC-GAG---GACGGCG-G-CGTGGTGA--C-CGT--GACCCAGGACTC---CTCCCTGCAGGACGGCG-AGTTC-A-TCTACAAGGTGAAGCTG-CGCGGCACCAAC-TTCCCCTC

*   $SW('TCGC', 'CTTAG')$ retourne un score de $1.5$ à la position $(3,5)$ et l'alignement

In [48]:
from IPython.display import HTML
HTML("<table align='left' style='font-family:Courier New'><tr><th>x:</th><th>TCG</th></tr><tr><th>y:</th><th>TAG</th></tr></table>")

x:,TCG
y:,TAG


*   $SW(AY678264^*, OQ870305^*)$ retourne un score de $342.1$ à la position $(708,717)$ et l'alignement

In [49]:
from IPython.display import HTML
HTML("<table width='300px' align='left' style='font-family:Courier New'><tr><th>x:</th><th nowrap='nowrap'>ATGGTGAGCAAGGGCGAGGAGGATAACATGGCCATCATCAAGGAGTTCATGCGCTTCAAGGTGC-A-CATGGAGGGCTCCGTGAACGGCCACGAGTTCGAGATCGAG---GGCGAGGGCGAGGGC--CGCC-CCTACGAGGGCACCCAGACCGC-CAAGCTGAAGGTG-ACCA-AGG---G-TGGCC---CCCT-GCCCTTCGCCT-GGGA-CATCCTGTCC--C--C-T-CAGTTCATGT-A-CGGCT-CCAAGGCCTACGTG-A--AGCAC--C--C--C--G-CCGACATCCCCG-A--CTAC-T--TGAAGCTG-TCCTTC--C--C-----CGA-GG--GCTTCAAGTGGGAGCG-CGTGATGAACTTCGAGGACGGCGGCGTGGTG-ACCG--T-GA-C-CCAGGAC-TC--CTCCCTGCAGGACGGCGAGTTCATCTACAAGGTG---AAGCTGCGCGGCACCAACTTCCCCT-CCGACGGCCCCGTA-ATGCA-GAAGAAGACCATGGGCTG--GGA-GGCCTCCTCCGAGCGGATGTACCCCGAGGA-CGGCGCC-CTGAAGGGCGAGATCAAGCAGA-GGCTGAAGC-TGAAGGACGGCGGCCACTACGACGCTGAGGTCAAGACCACCTACA-AGGCCAAGAAG-CCCGTGCAGCTGCCCGGC-GCCTACAACGTCAACATCAAGT-TG----GA-CATCACCTCCCACAACGAGGA-CTAC-A-C-CA---T-C-G-TGGAACAGTACG-AACGCGCCGAGGGCCGCCACTCCAC-CGGCGGCATGGACGAGCTGTACAAG</th></tr><tr><th>y:</th><th>ATGGTGAGCAAGGGCGAGGA-G----C-T-G--TTCA-C-CGG-GGTGGTGCCCATCCTGGT-CGAGC-TGGACGGCGACGTAAACGGCCACAAGTTC-AG--CGTGTCCGGCGAGGGCGAGGGCGATGCCACCTAC---GGCAAGCTGACC-CTGAAG-TTCATTTGCACCACCGGCAAGCTGCCCGTGCCCTGGCCC-AC-CCTCGTGACCACCCTGACCTACGGCGTGCAGTGC-T-TCAGCCGCTACCCCGACC-ACATGAAGCAGCACGACTTCTTCAAGTCCGCCATGCCCGAAGGCTACGTCCAGGAGC-GCACCATCTTCTTCAAGGACGACGGCAACTACAAGA-CCCGCGCCGAGGTGAAGTTCGAGGGCGACACCCTGGTGAACCGCATCGAGCTGAAGGGCATCGACTTCAAGGAGGACGGC-A--ACATC--C-TGGGGCACAAGCTG-G-AGTA-CAACTACAACAGCC-ACAACGTC-TATAT-CATG--GCCGA-CAA--GCAGAAGAACGG-CA--T-C-A-AGG-TGAACTTC-AAGATC--CGCCAC--AA---C---ATCGAG--GACGGC---AGCGTGCAGCTCGCCGACCACTACCA-GC--A-G--AACACC-CC--CATCGGCGACG--GCCCCGTGCTGCTGCCCGACAACC-ACTACCTGAGCACCCAGTCCGCCCTGAGCAA-A-GACCC-CAACGAGAAGC-GCGATCACATGGTCCTGCTGG---AGTTCGTGAC-CGCC----GCCGGGA-T-CACTC-TCGGCATGGACGAGCTGTACAAG</th></tr></table>")

x:,ATGGTGAGCAAGGGCGAGGAGGATAACATGGCCATCATCAAGGAGTTCATGCGCTTCAAGGTGC-A-CATGGAGGGCTCCGTGAACGGCCACGAGTTCGAGATCGAG---GGCGAGGGCGAGGGC--CGCC-CCTACGAGGGCACCCAGACCGC-CAAGCTGAAGGTG-ACCA-AGG---G-TGGCC---CCCT-GCCCTTCGCCT-GGGA-CATCCTGTCC--C--C-T-CAGTTCATGT-A-CGGCT-CCAAGGCCTACGTG-A--AGCAC--C--C--C--G-CCGACATCCCCG-A--CTAC-T--TGAAGCTG-TCCTTC--C--C-----CGA-GG--GCTTCAAGTGGGAGCG-CGTGATGAACTTCGAGGACGGCGGCGTGGTG-ACCG--T-GA-C-CCAGGAC-TC--CTCCCTGCAGGACGGCGAGTTCATCTACAAGGTG---AAGCTGCGCGGCACCAACTTCCCCT-CCGACGGCCCCGTA-ATGCA-GAAGAAGACCATGGGCTG--GGA-GGCCTCCTCCGAGCGGATGTACCCCGAGGA-CGGCGCC-CTGAAGGGCGAGATCAAGCAGA-GGCTGAAGC-TGAAGGACGGCGGCCACTACGACGCTGAGGTCAAGACCACCTACA-AGGCCAAGAAG-CCCGTGCAGCTGCCCGGC-GCCTACAACGTCAACATCAAGT-TG----GA-CATCACCTCCCACAACGAGGA-CTAC-A-C-CA---T-C-G-TGGAACAGTACG-AACGCGCCGAGGGCCGCCACTCCAC-CGGCGGCATGGACGAGCTGTACAAG
y:,ATGGTGAGCAAGGGCGAGGA-G----C-T-G--TTCA-C-CGG-GGTGGTGCCCATCCTGGT-CGAGC-TGGACGGCGACGTAAACGGCCACAAGTTC-AG--CGTGTCCGGCGAGGGCGAGGGCGATGCCACCTAC---GGCAAGCTGACC-CTGAAG-TTCATTTGCACCACCGGCAAGCTGCCCGTGCCCTGGCCC-AC-CCTCGTGACCACCCTGACCTACGGCGTGCAGTGC-T-TCAGCCGCTACCCCGACC-ACATGAAGCAGCACGACTTCTTCAAGTCCGCCATGCCCGAAGGCTACGTCCAGGAGC-GCACCATCTTCTTCAAGGACGACGGCAACTACAAGA-CCCGCGCCGAGGTGAAGTTCGAGGGCGACACCCTGGTGAACCGCATCGAGCTGAAGGGCATCGACTTCAAGGAGGACGGC-A--ACATC--C-TGGGGCACAAGCTG-G-AGTA-CAACTACAACAGCC-ACAACGTC-TATAT-CATG--GCCGA-CAA--GCAGAAGAACGG-CA--T-C-A-AGG-TGAACTTC-AAGATC--CGCCAC--AA---C---ATCGAG--GACGGC---AGCGTGCAGCTCGCCGACCACTACCA-GC--A-G--AACACC-CC--CATCGGCGACG--GCCCCGTGCTGCTGCCCGACAACC-ACTACCTGAGCACCCAGTCCGCCCTGAGCAA-A-GACCC-CAACGAGAAGC-GCGATCACATGGTCCTGCTGG---AGTTCGTGAC-CGCC----GCCGGGA-T-CACTC-TCGGCATGGACGAGCTGTACAAG


# Exercice 3 : Distribution des scores d’alignement pour des séquences aléatoires

Pour tester si un alignement reflète une réelle similarité biologique, on va évaluer la distribution des scores d’alignement pour des paires de séquences aléatoires.

Q1. En considérant deux séquences aléatoires de même taille N, où chaque nucléotide apparaît avec une probabilité uniforme de ¼, calculer le score moyen attendu pour une superposition sans trou dans le cas où une identité vaut +1 et une différence vaut 0.

```markdown
Le score pour chaque position identique est +1 et pour chaque position différente 0. Donc, le score moyen attendu pour la superposition sans trou est : N * 1/4 * 1 + N * 3/4 * 0 = N/4
```

Q2. La question précédente peut se resoudre analytiquement car on ne considère pas de trou. Pour étendre le résultat precedent à un alignement avec trous, on va se baser sur la simulation de séquences aleatoires.

Générez $R$ paires de séquences aléatoires  de tailles $N$ avec des probabilitées uniformes d'apparition de nucléotides $p_A = p_T = p_G = p_C = $ ¼. Affichez sous forme de violinplots les distribution des scores d'alignements entre chaque paire, obtenu par :
  1. un alignement sans trou (cf. Q1) ;
  2. un alignement local via Smith-Waterman (utilisez le code de l'exercice précédent)

Utilisez le schéma d'évaluation suivant :

In [ ]:
rmap = {"A": 0, "T": 1, "G": 2, "C": 3}
sigma = np.array([[1, -0.5, -0.5, -0.5],
                  [-0.5, 1, -0.5, -0.5],
                  [-0.5, -0.5, 1, -0.5],
                  [-0.5, -0.5, -0.5, 1]])
go =0
ge = 0.5

In [ ]:
#Votre code ici

Q3. Qu'observez-vous ?

```markdown
Votre réponse ici
```

Q4. Quelle conclusion peut-on en tirer sur la significativité d'un alignement ?

```markdown
Votre réponse ici
```